### Install (if needed) all prereqs below

In [1]:
#!pip install requests
#!pip install openai
#!pip install dotenv

In [2]:
import os
import requests
from dotenv import dotenv_values
import json
from openai import AzureOpenAI

# Create the env file based on the template.env provided. Put all key values, endpoints, etc. according to the template
env_name = "env.env" 
config = dotenv_values(env_name)


#### Using Whisper Model to Transcript a Call Center Call

In [3]:
# Configuration

# The Whisper model will be used to make the transcript
GPT4_WHISPER_KEY = config['openai_api_key_whisper']
GPT4_WHISPER_ENDPOINT = config['openai_completion_endpoint_whisper']
whisper_model_name = config['openai_api_whisper_model']
DEPLOYMENT_W_VERSION = config['openai_api_whisper_version']


AUDIO_PATH = "audio\Call1_separated_16k_health_insurance.wav"


# And we will use the multi-modal GPT4V to Analyze the content
GPT4V_KEY = config['openai_api_key_vision']
GPT4V_ENDPOINT = config['openai_completion_endpoint_vision']


client_w = AzureOpenAI(
  api_key = GPT4_WHISPER_KEY,  
  api_version = DEPLOYMENT_W_VERSION,
  azure_endpoint = GPT4_WHISPER_ENDPOINT
)


In [4]:
result = client_w.audio.transcriptions.create(
            file=open(AUDIO_PATH, "rb"),            
            model=whisper_model_name
        )

transcript = result.text

In [5]:
print(transcript)

Hello, thank you for calling Contoso. Who am I speaking with today? Hi, my name is Mary Rondo. I'm trying to enroll myself with Contoso. Hi, Mary. Are you calling because you need health insurance? Yes. Yeah, I'm calling to sign up for insurance. Great. If you can answer a few questions, we can get you signed up into JFE. Okay. So, what's your full name? So, Mary Beth Rondo. Last name is R like Romeo, O like ocean, N like Nancy, D like dog, and O like ocean again. Rondo. Got it. And what's the best callback number in case you get disconnected? I only have a cell phone, so I can give you that. Yep, that'll be fine. Sure. So, it's 234-554-9312. Got it. So, to confirm, it's 234-554-9312? Yep, that's right. Excellent. Let's get some additional information for your application. Do you have a job? Yes, I am self-employed. Okay. So, then you have a social security number as well? Yes, I do. Okay. And what is your social security number, please? Sure. So, it's 412-256789. Sorry, was that a 25 

In [6]:
headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

# Payload for the request
payload = {
  "enhancements": {
    "ocr": {
      "enabled": True
    },
    "grounding": {
      "enabled": True
    }
  },
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": """You are an AI assistant that will analyze a call center call
          
                     This is a Transcript between an agent and a customer conversation. Please identify each one.

                     Analyze if the agent asked the full name and the call back contact. Also verify if the agent asked the social security number.
                     Analyze if the agent provided information about applying to the health insurance and confirm that a sales agent will call them back
                     Verify if the agent asked the customer about their health plan preferences regarding the plan tier, like basic or premium
          
                     Analyze the customer prompt and generate a score of the general sentiment from 0 to 1, where 0 is negative sentment and 1 positive sentiment

                     Provide the results in the following JSON format without markdowns:

                     {
                     
                      callcenter_agent_id: 
                      customer_id:                      
                      tasks_asked_customer_full_name: yes/no
                      tasks_asked_callback_contact: yes/no
                      tasks_asked_social_security_number: yes/no
                      tasks_provided_applying_information: yes/no
                      tasks_asked_health_plan_tier_preference: yes/no
                      customer_sentiment_score: 

                     }
          
          """
        }
      ]
    },
    {
      "role": "user",
      "content": [
                {
          "type": "text",
          "text": "Here are some context for this call. The Agent ID is 021373 and the Customer ID is 980761"
        }
      ]
    },
    {
      "role": "user",
      "content": [
                {
          "type": "text",
          "text": "Transcript: " + transcript
        }
      ]
    }
  ],
  "temperature": 0,
  "top_p": 0.95,
  "max_tokens": 4096
}



# Send request
try:
    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")


result = response.json()

In [7]:
result = response.json()

return_value = result["choices"][0]["message"]["content"]

print(return_value)


{
  callcenter_agent_id: 021373
  customer_id: 980761                      
  tasks_asked_customer_full_name: yes
  tasks_asked_callback_contact: yes
  tasks_asked_social_security_number: yes
  tasks_provided_applying_information: yes
  tasks_asked_health_plan_tier_preference: no
  customer_sentiment_score: 0.9
}
